Note: I left the code to generate the htfs for each test system in the notebook, but if you don't want to re-generate, you can use the pickled htf that I saved (I left the full path in for you). I generated these htfs based after you added support for `lambda_{i}_bonds_old/new` and `scale_lambda_0_bonds`

In [17]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
import pickle
import copy

#############################################
# CONSTANTS
#############################################
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


In [18]:
def test_torsion_energies(htf, is_old=True, is_solvated=False, check_scale=False):
    htf_copy = copy.deepcopy(htf)
    
    # Get periodic torsion force and old/new positions
    system = htf._topology_proposal.old_system if is_old else htf._topology_proposal.new_system
    periodic_torsion_force = system.getForce(2) 
    positions = htf.old_positions(htf.hybrid_positions) if is_old else htf.new_positions(htf.hybrid_positions)
    
    # Get custom torsion force and hybrid positions
    torsion_force_index = 3 if is_solvated else 2
    hybrid_system = htf.hybrid_system
    custom_torsion_force = hybrid_system.getForce(torsion_force_index)
    hybrid_positions = htf.hybrid_positions

    # Remove nonbonded/exception forces -- this is needed because otherwise the force groups are messed up for the nonbonded forces, so all valence energies become super negative
    for i in range(hybrid_system.getNumForces(), hybrid_system.getNumForces() - 2, -1):
        hybrid_system.removeForce(i - 1)
    
    # Set global parameters
    lambda_old = 1 if is_old else 0
    lambda_new = 0 if is_old else 1
    for i in range(custom_torsion_force.getNumGlobalParameters()):
        if custom_torsion_force.getGlobalParameterName(i) == 'lambda_alchemical_torsions_old':
            custom_torsion_force.setGlobalParameterDefaultValue(i, lambda_old)
        elif custom_torsion_force.getGlobalParameterName(i) == 'lambda_alchemical_torsions_new':
            custom_torsion_force.setGlobalParameterDefaultValue(i, lambda_new)
            
    # Zero the unique old/new torsions in the custom torsion force
    hybrid_to_torsion_indices = htf._hybrid_to_new_torsion_indices if is_old else htf._hybrid_to_old_torsion_indices
    for hybrid_idx, idx in hybrid_to_torsion_indices.items():
        p1, p2, p3, p4, hybrid_params = custom_torsion_force.getTorsionParameters(hybrid_idx)
        hybrid_params = list(hybrid_params)
        hybrid_params[-1] *= 0
        hybrid_params[-2] *= 0
        custom_torsion_force.setTorsionParameters(hybrid_idx, p1, p2, p3, p4, hybrid_params)
    
    ## Get energy components of standard torsion force
    platform = configure_platform(REFERENCE_PLATFORM)
    thermostate_other = ThermodynamicState(system=system, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other = compute_potential_components(context_other, beta=beta)
    print(components_other)
    
    ## Get energy components of custom torsion force
    thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid = compute_potential_components(context_hybrid, beta=beta)
    print(components_hybrid)
    
    assert np.isclose([components_other[2][1]], [components_hybrid[2][1]])
    
    print("Success! Custom torsion force and standard torsion force energies are equal!")
    
    if check_scale:
        
        ## Get custom torsion force and hybrid positions
        torsion_force_index = 3 if is_solvated else 2
        hybrid_system = htf_copy.hybrid_system
        custom_torsion_force = hybrid_system.getForce(torsion_force_index)
        hybrid_positions = htf_copy.hybrid_positions
        
        # Remove nonbonded/exception forces -- this is needed because otherwise the force groups are messed up for the nonbonded forces, so all valence energies become super negative
        for i in range(hybrid_system.getNumForces(), hybrid_system.getNumForces() - 2, -1):
            hybrid_system.removeForce(i - 1)
        
        # Zero the unique old/new torsions in the custom torsion force
        hybrid_to_torsion_indices = htf_copy._hybrid_to_new_torsion_indices if is_old else htf_copy._hybrid_to_old_torsion_indices
        for hybrid_idx, idx in hybrid_to_torsion_indices.items():
            p1, p2, p3, p4, hybrid_params = custom_torsion_force.getTorsionParameters(hybrid_idx)
            hybrid_params = list(hybrid_params)
            hybrid_params[-1] *= 0
            hybrid_params[-2] *= 0
            custom_torsion_force.setTorsionParameters(hybrid_idx, p1, p2, p3, p4, hybrid_params)

        ## Get energy components of custom torsion force
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid)
        
        # Set `scale_lambda_{i}` to 0.5
        for i in range(custom_torsion_force.getNumGlobalParameters()):
            if custom_torsion_force.getGlobalParameterName(i) == 'lambda_rest_torsions':
                custom_torsion_force.setGlobalParameterDefaultValue(i, 0.5)
        
        ## Get energy components of custom torsion force iwth scaling
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid_scaled = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid_scaled)
        
        assert not np.isclose([components_hybrid[2][1]], [components_hybrid_scaled[2][1]])
        
        print("Success! Scaling the torsion force changes the energy")

# Alanine dipeptide in vacuum

### Test one alchemical region, no rest regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [19]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [20]:
torsion_force = htf.hybrid_system.getForce(2)
for i in range(torsion_force.getNumTorsions()):
    print(torsion_force.getTorsionParameters(i))

[0, 1, 4, 5, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.3472000000000004, 0.0)]
[0, 1, 4, 5, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 3.141592653589793, 0.0, 0.33472, 0.0)]
[1, 4, 6, 7, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.141592653589793, 0.0, 10.46, 0.0)]
[1, 4, 6, 8, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.141592653589793, 0.0, 10.46, 0.0)]
[2, 1, 4, 5, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.3472000000000004, 0.0)]
[2, 1, 4, 5, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 3.141592653589793, 0.0, 0.33472, 0.0)]
[3, 1, 4, 5, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.3472000000000004, 0.0)]
[3, 1, 4, 5, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 3.141592653589793, 0.0, 0.33472, 0.0)]
[4, 6, 8, 10, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.3472000000000004, 0.0)]
[4, 6, 8, 10, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 7.5312, 0.0)]
[4, 6, 8, 10, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0,

In [21]:
test_torsion_energies(htf)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0345517625414457), ('HarmonicAngleForce', 0.6072023625714801), ('PeriodicTorsionForce', 16.176828968049836), ('NonbondedForce', -39.186928227296285), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.104601379728185), ('CustomAngleForce', 226.17434542326083), ('CustomTorsionForce', 16.17683393823634), ('CustomNonbondedForce', -134.40418217693514), ('CustomNonbondedForce', 4.716822222623423), ('AndersenThermostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [22]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [23]:
torsion_force = htf.hybrid_system.getForce(2)
for i in range(torsion_force.getNumTorsions()):
    print(torsion_force.getTorsionParameters(i))

[0, 1, 4, 5, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.3472000000000004, 0.0)]
[0, 1, 4, 5, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 3.141592653589793, 0.0, 0.33472, 0.0)]
[1, 4, 6, 7, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.141592653589793, 0.0, 10.46, 0.0)]
[1, 4, 6, 8, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.141592653589793, 0.0, 10.46, 0.0)]
[2, 1, 4, 5, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.3472000000000004, 0.0)]
[2, 1, 4, 5, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 3.141592653589793, 0.0, 0.33472, 0.0)]
[3, 1, 4, 5, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.3472000000000004, 0.0)]
[3, 1, 4, 5, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 3.141592653589793, 0.0, 0.33472, 0.0)]
[4, 6, 8, 10, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.3472000000000004, 0.0)]
[4, 6, 8, 10, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 7.5312, 0.0)]
[4, 6, 8, 10, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0,

In [24]:
htf._hybrid_to_new_torsion_indices

{59: 17,
 60: 18,
 61: 19,
 62: 20,
 63: 21,
 64: 22,
 65: 23,
 66: 24,
 70: 28,
 71: 29,
 72: 30,
 73: 31,
 74: 32,
 75: 33,
 76: 34,
 80: 38,
 81: 39,
 88: 46,
 89: 47,
 90: 48,
 91: 49,
 92: 50,
 93: 51,
 94: 52,
 95: 53,
 99: 57,
 100: 58,
 101: 59,
 102: 60,
 103: 61,
 104: 62,
 105: 63,
 106: 64,
 107: 65,
 108: 66,
 109: 67}

In [25]:
test_torsion_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.104601379728185), ('HarmonicAngleForce', 226.17416732251837), ('PeriodicTorsionForce', 28.942507919755776), ('NonbondedForce', -32.888182721898666), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.104601379728185), ('CustomAngleForce', 226.17434542326083), ('CustomTorsionForce', 28.94251268471297), ('CustomNonbondedForce', -134.40418217693514), ('CustomNonbondedForce', 4.716822222623423), ('AndersenThermostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


### Test one alchemical region, one rest region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [26]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [27]:
test_torsion_energies(htf, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0345517625414457), ('HarmonicAngleForce', 0.6072023625714801), ('PeriodicTorsionForce', 16.176828968049836), ('NonbondedForce', -39.186928227296285), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.161946421456782), ('CustomAngleForce', 276.29676907348716), ('CustomTorsionForce', 16.17683393823634), ('CustomNonbondedForce', -134.40418217693514), ('CustomNonbondedForce', 4.716822222623423), ('AndersenThermostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.161946421456782), ('CustomAngleForce', 276.29676907348716), ('CustomTorsionForce', 16.17683393823634), ('CustomNonbondedForce', -134.40418217693514), ('CustomNonbondedForce', 4.716822222623423), ('AndersenT

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [28]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [29]:
test_torsion_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.161946421456781), ('HarmonicAngleForce', 276.2965909727446), ('PeriodicTorsionForce', 29.734502023335214), ('NonbondedForce', 186.47419623613024), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.161946421456782), ('CustomAngleForce', 276.29676907348716), ('CustomTorsionForce', 29.734507572757877), ('CustomNonbondedForce', -134.40418217693514), ('CustomNonbondedForce', 4.716822222623423), ('AndersenThermostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


# Alanine dipeptide in solvent

### Test one alchemical region, no rest regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [30]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [31]:
htf.hybrid_system.getForces()

[<openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2af847f224e0> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2af847f227e0> >,
 <openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2af847f22360> >,
 <openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2af847f223f0> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2af847f221e0> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2af847f22270> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2af847f22060> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2af847f220f0> >]

In [32]:
test_torsion_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03455176254144657), ('HarmonicAngleForce', 0.6072023625714819), ('PeriodicTorsionForce', 16.176828968049854), ('NonbondedForce', -902.1313615859381), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.10424684793482342), ('CustomAngleForce', 253.67923506003564), ('CustomTorsionForce', 16.17683393823634), ('CustomNonbondedForce', -7419.6545231897835), ('CustomNonbondedForce', 7020.872908390913), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [33]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [34]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.10424684793482342), ('HarmonicAngleForce', 253.6790569592932), ('PeriodicTorsionForce', 28.668625160559667), ('NonbondedForce', -891.8052080614868), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.10424684793482342), ('CustomAngleForce', 253.67923506003564), ('CustomTorsionForce', 28.66862933209474), ('CustomNonbondedForce', -7419.6545231897835), ('CustomNonbondedForce', 7020.872908390913), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


### Test one alchemical region, one rest region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [35]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [36]:
test_torsion_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03455176254144657), ('HarmonicAngleForce', 0.6072023625714819), ('PeriodicTorsionForce', 16.176828968049854), ('NonbondedForce', -999.1575928937541), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.06438765035508309), ('CustomAngleForce', 7.9052489977321105), ('CustomTorsionForce', 16.17683393823634), ('CustomNonbondedForce', -7425.786600236032), ('CustomNonbondedForce', 6996.413691009773), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.06438765035508309), ('CustomAngleForce', 7.9052489977321105), ('CustomTorsionForce', 16.17683393823634), ('CustomNonbondedForce', -7425.78660023603

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [37]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [38]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.06438765035508309), ('HarmonicAngleForce', 7.90507089698963), ('PeriodicTorsionForce', 24.83642918340053), ('NonbondedForce', -1029.6551062388573), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.06438765035508309), ('CustomAngleForce', 7.9052489977321105), ('CustomTorsionForce', 24.836434152761374), ('CustomNonbondedForce', -7425.786600236032), ('CustomNonbondedForce', 6996.413691009771), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


# CDK2 transformation in solvent

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [39]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [40]:
test_torsion_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.5773624247683875), ('HarmonicAngleForce', 29.004860118481027), ('PeriodicTorsionForce', 14.976236339333706), ('NonbondedForce', 1539.5025935138624), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 29.05030818391434), ('CustomTorsionForce', 14.976242995104998), ('CustomNonbondedForce', -11540.561656333028), ('CustomNonbondedForce', 14285.141811021627), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [41]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [42]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.489378129340331), ('HarmonicAngleForce', 29.00425408642172), ('PeriodicTorsionForce', 14.995423633625117), ('NonbondedForce', 1551.0561832017609), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 29.05030818391434), ('CustomTorsionForce', 14.995432385454839), ('CustomNonbondedForce', -11540.561656333031), ('CustomNonbondedForce', 14285.141811021627), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [43]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [44]:
torsion_force = htf.hybrid_system.getForce(3)
for i in range(torsion_force.getNumTorsions()):
    print(torsion_force.getTorsionParameters(i))

[0, 13, 14, 15, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.141592653589793, 0.0, 20.0832, 0.0)]
[0, 13, 14, 16, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.141592653589793, 0.0, 20.0832, 0.0)]
[0, 20, 19, 16, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.141592653589793, 0.0, 20.0832, 0.0)]
[0, 20, 19, 18, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.141592653589793, 0.0, 20.0832, 0.0)]
[0, 21, 22, 12, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.141592653589793, 0.0, 4.3932, 0.0)]
[0, 21, 22, 23, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.141592653589793, 0.0, 4.3932, 0.0)]
[1, 17, 16, 14, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.141592653589793, 0.0, 20.0832, 0.0)]
[1, 17, 16, 19, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.141592653589793, 0.0, 20.0832, 0.0)]
[1, 18, 19, 16, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.141592653589793, 0.0, 2.522952, 0.0)]
[1, 18, 19, 20, (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.141592653589793, 0.0, 1.2552

In [45]:
test_torsion_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.5773624247683875), ('HarmonicAngleForce', 29.004860118481027), ('PeriodicTorsionForce', 14.976236339333706), ('NonbondedForce', 2176.8376850868117), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 29.007568667470167), ('CustomTorsionForce', 14.976242995104998), ('CustomNonbondedForce', -11630.05784157921), ('CustomNonbondedForce', 15162.96932890813), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 29.007568667470167), ('CustomTorsionForce', 14.976242995104998), ('CustomNonbondedForce', -11630.05784157921

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [46]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [47]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.489378129340331), ('HarmonicAngleForce', 28.96151456997755), ('PeriodicTorsionForce', 14.977081953208152), ('NonbondedForce', 2187.0647485063373), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 29.007568667470167), ('CustomTorsionForce', 14.977088857121053), ('CustomNonbondedForce', -11630.05784157921), ('CustomNonbondedForce', 15162.969328908126), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


# 8mer in solvent

### Test one alchemical region, no rest regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [37]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [38]:
test_torsion_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153172), ('HarmonicAngleForce', 12.445685055931843), ('PeriodicTorsionForce', 156.2435090637808), ('NonbondedForce', -9963.845239675748), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153172), ('CustomAngleForce', 12.445685055931843), ('CustomTorsionForce', 156.24352300681832), ('CustomNonbondedForce', -27470.062373416964), ('CustomNonbondedForce', 18758.303916127312), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [39]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)


In [40]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 9.361655470384113), ('HarmonicAngleForce', 14.18827616458745), ('PeriodicTorsionForce', 174.97365990635137), ('NonbondedForce', -9771.178805994987), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153172), ('CustomAngleForce', 12.445685055931843), ('CustomTorsionForce', 174.97367915013456), ('CustomNonbondedForce', -27470.062373416964), ('CustomNonbondedForce', 18758.303916127312), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


### Test one alchemical region, one rest region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [55]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [56]:
test_torsion_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153172), ('HarmonicAngleForce', 12.445685055931843), ('PeriodicTorsionForce', 156.2435090637808), ('NonbondedForce', -10206.31135544179), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153172), ('CustomAngleForce', 12.445685055931843), ('CustomTorsionForce', 156.24352300681832), ('CustomNonbondedForce', -27503.63689266387), ('CustomNonbondedForce', 18788.870693529905), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153172), ('CustomAngleForce', 12.445685055931843), ('CustomTorsionForce', 156.24352300681832), ('CustomNonbondedForce', -27503.63689266388), 

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [43]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [44]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 8.6173683680896), ('HarmonicAngleForce', 27.500492144184243), ('PeriodicTorsionForce', 163.92624538706607), ('NonbondedForce', 1477052.4243630853), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153172), ('CustomAngleForce', 12.445685055931843), ('CustomTorsionForce', 163.92626684142152), ('CustomNonbondedForce', -27503.63689266388), ('CustomNonbondedForce', 18788.87069352991), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


## Barnase:barstar (no naked charge fix)

### Test one alchemical region, no scale regions

#### Old system PeroidicTorsionForce vs hybrid system CustomTorsionForce

In [48]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [49]:
test_torsion_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 569.5926176486894), ('HarmonicAngleForce', 1323.9862513751366), ('PeriodicTorsionForce', 3795.5154898495653), ('NonbondedForce', 57388553.61774712), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.5926176486894), ('CustomAngleForce', 1524.3329076618577), ('CustomTorsionForce', 3795.515748682387), ('CustomNonbondedForce', -171231.54069802086), ('CustomNonbondedForce', 57571153.000987224), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicTorsionForce vs hybrid system CustomTorsionForce

In [50]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [51]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 569.2585424303023), ('HarmonicAngleForce', 1522.6319866241859), ('PeriodicTorsionForce', 3785.2911016376443), ('NonbondedForce', 57388631.747133516), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.5926176486894), ('CustomAngleForce', 1524.3329076618577), ('CustomTorsionForce', 3785.291360636841), ('CustomNonbondedForce', -171231.54069802086), ('CustomNonbondedForce', 57571153.0009872), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


### Test one alchemical region, one scale regions

#### Old system HarmonicTorsionForce vs hybrid system CustomTorsionForce

In [52]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [53]:
test_torsion_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 569.5926176486894), ('HarmonicAngleForce', 1323.9862513751366), ('PeriodicTorsionForce', 3795.5154898495653), ('NonbondedForce', 57387942.55770265), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.5926176486894), ('CustomAngleForce', 1531.9788896260216), ('CustomTorsionForce', 3795.515748682387), ('CustomNonbondedForce', -171234.61335319918), ('CustomNonbondedForce', 57570281.796158895), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.5926176486894), ('CustomAngleForce', 1531.9788896260216), ('CustomTorsionForce', 3795.515748682387), ('CustomNonbondedForce', -171234.61335319918), 

#### New system HarmonicTorsionForce vs hybrid system CustomTorsionForce

In [54]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [55]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 569.2585424303023), ('HarmonicAngleForce', 1530.2779685883497), ('PeriodicTorsionForce', 3785.2906533191526), ('NonbondedForce', 57388021.86258256), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.5926176486894), ('CustomAngleForce', 1531.9788896260216), ('CustomTorsionForce', 3785.2909123187596), ('CustomNonbondedForce', -171234.61335319918), ('CustomNonbondedForce', 57570281.79615893), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


In [4]:
htf._topology_proposal.old_system.getForces()

[<openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2ad06e2cc1b0> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2ad06e2cc090> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2ad06e2ccb40> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2ad06e2cc2d0> >,
 <openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2ad06e2cc4b0> >]

In [5]:
old_system_torsion_force = htf._topology_proposal.old_system.getForce(2)

In [8]:
_old_to_hybrid_map = {}
hybrid_system = openmm.System()
for particle_idx in range(htf._topology_proposal.n_atoms_old):
    particle_mass = htf._topology_proposal.old_system.getParticleMass(particle_idx)
    hybrid_idx = hybrid_system.addParticle(particle_mass)
    _old_to_hybrid_map[particle_idx] = hybrid_idx


In [20]:
old_term_collector = {}
for term_idx in range(old_system_torsion_force.getNumTorsions()):
    p1, p2, p3, p4, periodicity, phase, k = old_system_torsion_force.getTorsionParameters(term_idx) # Grab the parameters
    hybrid_p1, hybrid_p2, hybrid_p3, hybrid_p4 = _old_to_hybrid_map[p1], _old_to_hybrid_map[p2], _old_to_hybrid_map[p3], _old_to_hybrid_map[p4] # Make hybrid indices
    
    def _is_proper(p1, p2, p3, p4):
        bond_pairs = [(bond[0].index, bond[1].index)for bond in htf._topology_proposal.old_topology.bonds()]
        found = 0
        for pair in [(p1, p2), (p2, p3), (p3, p4)]:
            if (pair[0], pair[1]) in bond_pairs or (pair[1], pair[0]) in bond_pairs:
                found += 1
        return True if found == 3 else False

    if not _is_proper(p1, p2, p3, p4):
        print("FOUND improper: ", p1, p2, p3, p4)
    
    if hybrid_p1 < hybrid_p4:
        sorted_indices = tuple([hybrid_p1, hybrid_p2, hybrid_p3, hybrid_p4]) 
    else:
#         print("og order: ", hybrid_p1, hybrid_p2, hybrid_p3, hybrid_p4)
        sorted_indices = tuple([hybrid_p4, hybrid_p3, hybrid_p2, hybrid_p1])
#         print("swapping the order: ", sorted_indices)
    if sorted_indices in old_term_collector.keys():
        # It _is_ the case that some torsions have the same particle indices...
#         old_term_collector[sorted_indices].append([term_idx, periodicity, phase, k])
#         print("adding to existing key: ", sorted_indices, [term_idx, periodicity, phase, k])
        pass
    else:
        # Make this a nested list to hold multiple terms
        old_term_collector[sorted_indices] = [[term_idx, periodicity, phase, k]]


FOUND improper:  0 6 1 2
FOUND improper:  1 7 6 15
FOUND improper:  7 28 8 9
FOUND improper:  8 29 28 37
FOUND improper:  29 50 30 31
FOUND improper:  30 51 50 55
FOUND improper:  51 60 52 53
FOUND improper:  52 61 60 67
FOUND improper:  61 76 62 63
FOUND improper:  62 77 76 84
FOUND improper:  77 95 78 79
FOUND improper:  78 96 95 103
FOUND improper:  96 109 97 98
FOUND improper:  99 102 100 101
FOUND improper:  100 107 102 108
FOUND improper:  97 110 109 113
FOUND improper:  110 116 111 112
FOUND improper:  111 117 116 125
FOUND improper:  117 131 118 119
FOUND improper:  121 123 122 124
FOUND improper:  118 132 131 140
FOUND improper:  132 148 133 134
FOUND improper:  136 139 137 138
FOUND improper:  137 146 139 147
FOUND improper:  133 149 148 156
FOUND improper:  149 167 150 151
FOUND improper:  150 168 167 178
FOUND improper:  168 191 169 170
FOUND improper:  173 175 174 186
FOUND improper:  174 176 175 177
FOUND improper:  175 187 176 188
FOUND improper:  175 189 177 190
FOUND i

FOUND improper:  1217 1229 1218 1230
FOUND improper:  1217 1231 1219 1232
FOUND improper:  1211 1234 1233 1242
FOUND improper:  1234 1248 1235 1236
FOUND improper:  1238 1240 1239 1241
FOUND improper:  1235 1249 1248 1253
FOUND improper:  1249 1258 1250 1251
FOUND improper:  1250 1259 1258 1267
FOUND improper:  1259 1280 1260 1261
FOUND improper:  1260 1281 1280 1285
FOUND improper:  1281 1290 1282 1283
FOUND improper:  1282 1291 1290 1299
FOUND improper:  1291 1305 1292 1293
FOUND improper:  1295 1297 1296 1298
FOUND improper:  1292 1306 1305 1309
FOUND improper:  1306 1312 1307 1308
FOUND improper:  1307 1313 1312 1317
FOUND improper:  1313 1322 1314 1315
FOUND improper:  1314 1323 1322 1330
FOUND improper:  1323 1334 1324 1325
FOUND improper:  1326 1328 1327 1329
FOUND improper:  1324 1335 1334 1342
FOUND improper:  1335 1353 1336 1337
FOUND improper:  1336 1354 1353 1360
FOUND improper:  1354 1367 1355 1356
FOUND improper:  1355 1368 1367 1375
FOUND improper:  1368 1386 1369 1370
F

FOUND improper:  2681 2694 2693 2707
FOUND improper:  2694 2717 2695 2696
FOUND improper:  2700 2697 2698 2699
FOUND improper:  2698 2701 2699 2711
FOUND improper:  2699 2702 2701 2712
FOUND improper:  2705 2700 2703 2713
FOUND improper:  2702 2706 2704 2714
FOUND improper:  2703 2706 2705 2715
FOUND improper:  2704 2705 2706 2716
FOUND improper:  2695 2718 2717 2728
FOUND improper:  2718 2741 2719 2720
FOUND improper:  2723 2725 2724 2736
FOUND improper:  2724 2726 2725 2727
FOUND improper:  2725 2737 2726 2738
FOUND improper:  2725 2739 2727 2740
FOUND improper:  2719 2742 2741 2750
FOUND improper:  2742 2756 2743 2744
FOUND improper:  2746 2748 2747 2749
FOUND improper:  2743 2757 2756 2761
FOUND improper:  2757 2766 2758 2759
FOUND improper:  2758 2767 2766 2774
FOUND improper:  2767 2778 2768 2769
FOUND improper:  2770 2772 2771 2773
FOUND improper:  2768 2779 2778 2786
FOUND improper:  2779 2797 2780 2781
FOUND improper:  2780 2798 2797 2805
FOUND improper:  2798 2811 2799 2800
F

In [17]:
for atom in htf._topology_proposal.old_topology.atoms():
    print(atom.index, atom)

0 <Atom 0 (CH3) of chain 0 residue 0 (ACE)>
1 <Atom 1 (C) of chain 0 residue 0 (ACE)>
2 <Atom 2 (O) of chain 0 residue 0 (ACE)>
3 <Atom 3 (H1) of chain 0 residue 0 (ACE)>
4 <Atom 4 (H2) of chain 0 residue 0 (ACE)>
5 <Atom 5 (H3) of chain 0 residue 0 (ACE)>
6 <Atom 6 (N) of chain 0 residue 1 (LYS)>
7 <Atom 7 (CA) of chain 0 residue 1 (LYS)>
8 <Atom 8 (C) of chain 0 residue 1 (LYS)>
9 <Atom 9 (O) of chain 0 residue 1 (LYS)>
10 <Atom 10 (CB) of chain 0 residue 1 (LYS)>
11 <Atom 11 (CG) of chain 0 residue 1 (LYS)>
12 <Atom 12 (CD) of chain 0 residue 1 (LYS)>
13 <Atom 13 (CE) of chain 0 residue 1 (LYS)>
14 <Atom 14 (NZ) of chain 0 residue 1 (LYS)>
15 <Atom 15 (H2) of chain 0 residue 1 (LYS)>
16 <Atom 16 (HA) of chain 0 residue 1 (LYS)>
17 <Atom 17 (HB3) of chain 0 residue 1 (LYS)>
18 <Atom 18 (HB2) of chain 0 residue 1 (LYS)>
19 <Atom 19 (HG3) of chain 0 residue 1 (LYS)>
20 <Atom 20 (HG2) of chain 0 residue 1 (LYS)>
21 <Atom 21 (HD3) of chain 0 residue 1 (LYS)>
22 <Atom 22 (HD2) of chain 0 

1450 <Atom 1450 (O) of chain 1 residue 94 (HOH)>
1451 <Atom 1451 (H1) of chain 1 residue 94 (HOH)>
1452 <Atom 1452 (H2) of chain 1 residue 94 (HOH)>
1453 <Atom 1453 (O) of chain 1 residue 95 (HOH)>
1454 <Atom 1454 (H1) of chain 1 residue 95 (HOH)>
1455 <Atom 1455 (H2) of chain 1 residue 95 (HOH)>
1456 <Atom 1456 (O) of chain 1 residue 96 (HOH)>
1457 <Atom 1457 (H1) of chain 1 residue 96 (HOH)>
1458 <Atom 1458 (H2) of chain 1 residue 96 (HOH)>
1459 <Atom 1459 (O) of chain 1 residue 97 (HOH)>
1460 <Atom 1460 (H1) of chain 1 residue 97 (HOH)>
1461 <Atom 1461 (H2) of chain 1 residue 97 (HOH)>
1462 <Atom 1462 (O) of chain 1 residue 98 (HOH)>
1463 <Atom 1463 (H1) of chain 1 residue 98 (HOH)>
1464 <Atom 1464 (H2) of chain 1 residue 98 (HOH)>
1465 <Atom 1465 (O) of chain 1 residue 99 (HOH)>
1466 <Atom 1466 (H1) of chain 1 residue 99 (HOH)>
1467 <Atom 1467 (H2) of chain 1 residue 99 (HOH)>
1468 <Atom 1468 (O) of chain 1 residue 100 (HOH)>
1469 <Atom 1469 (H1) of chain 1 residue 100 (HOH)>
1470 

3200 <Atom 3200 (CG) of chain 2 residue 267 (ASP)>
3201 <Atom 3201 (OD1) of chain 2 residue 267 (ASP)>
3202 <Atom 3202 (OD2) of chain 2 residue 267 (ASP)>
3203 <Atom 3203 (H) of chain 2 residue 267 (ASP)>
3204 <Atom 3204 (HA) of chain 2 residue 267 (ASP)>
3205 <Atom 3205 (HB3) of chain 2 residue 267 (ASP)>
3206 <Atom 3206 (HB2) of chain 2 residue 267 (ASP)>
3207 <Atom 3207 (N) of chain 2 residue 268 (HIS)>
3208 <Atom 3208 (CA) of chain 2 residue 268 (HIS)>
3209 <Atom 3209 (C) of chain 2 residue 268 (HIS)>
3210 <Atom 3210 (O) of chain 2 residue 268 (HIS)>
3211 <Atom 3211 (CB) of chain 2 residue 268 (HIS)>
3212 <Atom 3212 (CG) of chain 2 residue 268 (HIS)>
3213 <Atom 3213 (ND1) of chain 2 residue 268 (HIS)>
3214 <Atom 3214 (CD2) of chain 2 residue 268 (HIS)>
3215 <Atom 3215 (CE1) of chain 2 residue 268 (HIS)>
3216 <Atom 3216 (NE2) of chain 2 residue 268 (HIS)>
3217 <Atom 3217 (H) of chain 2 residue 268 (HIS)>
3218 <Atom 3218 (HA) of chain 2 residue 268 (HIS)>
3219 <Atom 3219 (HB3) of cha

4950 <Atom 4950 (O) of chain 4 residue 801 (HOH)>
4951 <Atom 4951 (H1) of chain 4 residue 801 (HOH)>
4952 <Atom 4952 (H2) of chain 4 residue 801 (HOH)>
4953 <Atom 4953 (O) of chain 4 residue 802 (HOH)>
4954 <Atom 4954 (H1) of chain 4 residue 802 (HOH)>
4955 <Atom 4955 (H2) of chain 4 residue 802 (HOH)>
4956 <Atom 4956 (O) of chain 4 residue 803 (HOH)>
4957 <Atom 4957 (H1) of chain 4 residue 803 (HOH)>
4958 <Atom 4958 (H2) of chain 4 residue 803 (HOH)>
4959 <Atom 4959 (O) of chain 4 residue 804 (HOH)>
4960 <Atom 4960 (H1) of chain 4 residue 804 (HOH)>
4961 <Atom 4961 (H2) of chain 4 residue 804 (HOH)>
4962 <Atom 4962 (O) of chain 4 residue 805 (HOH)>
4963 <Atom 4963 (H1) of chain 4 residue 805 (HOH)>
4964 <Atom 4964 (H2) of chain 4 residue 805 (HOH)>
4965 <Atom 4965 (O) of chain 4 residue 806 (HOH)>
4966 <Atom 4966 (H1) of chain 4 residue 806 (HOH)>
4967 <Atom 4967 (H2) of chain 4 residue 806 (HOH)>
4968 <Atom 4968 (O) of chain 4 residue 807 (HOH)>
4969 <Atom 4969 (H1) of chain 4 residu

7199 <Atom 7199 (H2) of chain 4 residue 1550 (HOH)>
7200 <Atom 7200 (O) of chain 4 residue 1551 (HOH)>
7201 <Atom 7201 (H1) of chain 4 residue 1551 (HOH)>
7202 <Atom 7202 (H2) of chain 4 residue 1551 (HOH)>
7203 <Atom 7203 (O) of chain 4 residue 1552 (HOH)>
7204 <Atom 7204 (H1) of chain 4 residue 1552 (HOH)>
7205 <Atom 7205 (H2) of chain 4 residue 1552 (HOH)>
7206 <Atom 7206 (O) of chain 4 residue 1553 (HOH)>
7207 <Atom 7207 (H1) of chain 4 residue 1553 (HOH)>
7208 <Atom 7208 (H2) of chain 4 residue 1553 (HOH)>
7209 <Atom 7209 (O) of chain 4 residue 1554 (HOH)>
7210 <Atom 7210 (H1) of chain 4 residue 1554 (HOH)>
7211 <Atom 7211 (H2) of chain 4 residue 1554 (HOH)>
7212 <Atom 7212 (O) of chain 4 residue 1555 (HOH)>
7213 <Atom 7213 (H1) of chain 4 residue 1555 (HOH)>
7214 <Atom 7214 (H2) of chain 4 residue 1555 (HOH)>
7215 <Atom 7215 (O) of chain 4 residue 1556 (HOH)>
7216 <Atom 7216 (H1) of chain 4 residue 1556 (HOH)>
7217 <Atom 7217 (H2) of chain 4 residue 1556 (HOH)>
7218 <Atom 7218 (O

...

21198 <Atom 21198 (O) of chain 4 residue 6217 (HOH)>
21199 <Atom 21199 (H1) of chain 4 residue 6217 (HOH)>
21200 <Atom 21200 (H2) of chain 4 residue 6217 (HOH)>
21201 <Atom 21201 (O) of chain 4 residue 6218 (HOH)>
21202 <Atom 21202 (H1) of chain 4 residue 6218 (HOH)>
21203 <Atom 21203 (H2) of chain 4 residue 6218 (HOH)>
21204 <Atom 21204 (O) of chain 4 residue 6219 (HOH)>
21205 <Atom 21205 (H1) of chain 4 residue 6219 (HOH)>
21206 <Atom 21206 (H2) of chain 4 residue 6219 (HOH)>
21207 <Atom 21207 (O) of chain 4 residue 6220 (HOH)>
21208 <Atom 21208 (H1) of chain 4 residue 6220 (HOH)>
21209 <Atom 21209 (H2) of chain 4 residue 6220 (HOH)>
21210 <Atom 21210 (O) of chain 4 residue 6221 (HOH)>
21211 <Atom 21211 (H1) of chain 4 residue 6221 (HOH)>
21212 <Atom 21212 (H2) of chain 4 residue 6221 (HOH)>
21213 <Atom 21213 (O) of chain 4 residue 6222 (HOH)>
21214 <Atom 21214 (H1) of chain 4 residue 6222 (HOH)>
21215 <Atom 21215 (H2) of chain 4 residue 6222 (HOH)>
21216 <Atom 21216 (O) of chain 4 r

23698 <Atom 23698 (H1) of chain 4 residue 7050 (HOH)>
23699 <Atom 23699 (H2) of chain 4 residue 7050 (HOH)>
23700 <Atom 23700 (O) of chain 4 residue 7051 (HOH)>
23701 <Atom 23701 (H1) of chain 4 residue 7051 (HOH)>
23702 <Atom 23702 (H2) of chain 4 residue 7051 (HOH)>
23703 <Atom 23703 (O) of chain 4 residue 7052 (HOH)>
23704 <Atom 23704 (H1) of chain 4 residue 7052 (HOH)>
23705 <Atom 23705 (H2) of chain 4 residue 7052 (HOH)>
23706 <Atom 23706 (O) of chain 4 residue 7053 (HOH)>
23707 <Atom 23707 (H1) of chain 4 residue 7053 (HOH)>
23708 <Atom 23708 (H2) of chain 4 residue 7053 (HOH)>
23709 <Atom 23709 (O) of chain 4 residue 7054 (HOH)>
23710 <Atom 23710 (H1) of chain 4 residue 7054 (HOH)>
23711 <Atom 23711 (H2) of chain 4 residue 7054 (HOH)>
23712 <Atom 23712 (O) of chain 4 residue 7055 (HOH)>
23713 <Atom 23713 (H1) of chain 4 residue 7055 (HOH)>
23714 <Atom 23714 (H2) of chain 4 residue 7055 (HOH)>
23715 <Atom 23715 (O) of chain 4 residue 7056 (HOH)>
23716 <Atom 23716 (H1) of chain 4 

26197 <Atom 26197 (H1) of chain 4 residue 7883 (HOH)>
26198 <Atom 26198 (H2) of chain 4 residue 7883 (HOH)>
26199 <Atom 26199 (O) of chain 4 residue 7884 (HOH)>
26200 <Atom 26200 (H1) of chain 4 residue 7884 (HOH)>
26201 <Atom 26201 (H2) of chain 4 residue 7884 (HOH)>
26202 <Atom 26202 (O) of chain 4 residue 7885 (HOH)>
26203 <Atom 26203 (H1) of chain 4 residue 7885 (HOH)>
26204 <Atom 26204 (H2) of chain 4 residue 7885 (HOH)>
26205 <Atom 26205 (O) of chain 4 residue 7886 (HOH)>
26206 <Atom 26206 (H1) of chain 4 residue 7886 (HOH)>
26207 <Atom 26207 (H2) of chain 4 residue 7886 (HOH)>
26208 <Atom 26208 (O) of chain 4 residue 7887 (HOH)>
26209 <Atom 26209 (H1) of chain 4 residue 7887 (HOH)>
26210 <Atom 26210 (H2) of chain 4 residue 7887 (HOH)>
26211 <Atom 26211 (O) of chain 4 residue 7888 (HOH)>
26212 <Atom 26212 (H1) of chain 4 residue 7888 (HOH)>
26213 <Atom 26213 (H2) of chain 4 residue 7888 (HOH)>
26214 <Atom 26214 (O) of chain 4 residue 7889 (HOH)>
26215 <Atom 26215 (H1) of chain 4 

28697 <Atom 28697 (H2) of chain 4 residue 8716 (HOH)>
28698 <Atom 28698 (O) of chain 4 residue 8717 (HOH)>
28699 <Atom 28699 (H1) of chain 4 residue 8717 (HOH)>
28700 <Atom 28700 (H2) of chain 4 residue 8717 (HOH)>
28701 <Atom 28701 (O) of chain 4 residue 8718 (HOH)>
28702 <Atom 28702 (H1) of chain 4 residue 8718 (HOH)>
28703 <Atom 28703 (H2) of chain 4 residue 8718 (HOH)>
28704 <Atom 28704 (O) of chain 4 residue 8719 (HOH)>
28705 <Atom 28705 (H1) of chain 4 residue 8719 (HOH)>
28706 <Atom 28706 (H2) of chain 4 residue 8719 (HOH)>
28707 <Atom 28707 (O) of chain 4 residue 8720 (HOH)>
28708 <Atom 28708 (H1) of chain 4 residue 8720 (HOH)>
28709 <Atom 28709 (H2) of chain 4 residue 8720 (HOH)>
28710 <Atom 28710 (O) of chain 4 residue 8721 (HOH)>
28711 <Atom 28711 (H1) of chain 4 residue 8721 (HOH)>
28712 <Atom 28712 (H2) of chain 4 residue 8721 (HOH)>
28713 <Atom 28713 (O) of chain 4 residue 8722 (HOH)>
28714 <Atom 28714 (H1) of chain 4 residue 8722 (HOH)>
28715 <Atom 28715 (H2) of chain 4 

31197 <Atom 31197 (O) of chain 4 residue 9550 (HOH)>
31198 <Atom 31198 (H1) of chain 4 residue 9550 (HOH)>
31199 <Atom 31199 (H2) of chain 4 residue 9550 (HOH)>
31200 <Atom 31200 (O) of chain 4 residue 9551 (HOH)>
31201 <Atom 31201 (H1) of chain 4 residue 9551 (HOH)>
31202 <Atom 31202 (H2) of chain 4 residue 9551 (HOH)>
31203 <Atom 31203 (O) of chain 4 residue 9552 (HOH)>
31204 <Atom 31204 (H1) of chain 4 residue 9552 (HOH)>
31205 <Atom 31205 (H2) of chain 4 residue 9552 (HOH)>
31206 <Atom 31206 (O) of chain 4 residue 9553 (HOH)>
31207 <Atom 31207 (H1) of chain 4 residue 9553 (HOH)>
31208 <Atom 31208 (H2) of chain 4 residue 9553 (HOH)>
31209 <Atom 31209 (O) of chain 4 residue 9554 (HOH)>
31210 <Atom 31210 (H1) of chain 4 residue 9554 (HOH)>
31211 <Atom 31211 (H2) of chain 4 residue 9554 (HOH)>
31212 <Atom 31212 (O) of chain 4 residue 9555 (HOH)>
31213 <Atom 31213 (H1) of chain 4 residue 9555 (HOH)>
31214 <Atom 31214 (H2) of chain 4 residue 9555 (HOH)>
31215 <Atom 31215 (O) of chain 4 r

In [ ]:
# TO DO: actually test the scaling of the energies matches